In [1]:
# !pip install scikit-learn numpy pandas pyarrow

In [1]:
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import pickle

In [2]:
!curl https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet -o yellow_tripdata_2023-01.parquet
!curl https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet -o yellow_tripdata_2023-02.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.4M  100 45.4M    0     0   9.9M      0  0:00:04  0:00:04 --:--:--  9.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.5M  100 45.5M    0     0  10.1M      0  0:00:04  0:00:04 --:--:-- 10.4M


In [3]:
data_j= pd.read_parquet('yellow_tripdata_2023-01.parquet')
data_f = pd.read_parquet('yellow_tripdata_2023-02.parquet')

In [5]:
len(data_j.columns)

19

In [6]:
data_j['duration_min'] = (data_j['tpep_dropoff_datetime'] - data_j['tpep_pickup_datetime']) / pd.Timedelta(minutes=1)

In [7]:
data_j['duration_min'].std()

42.59435124195458

In [8]:
outlier_mask = ((data_j['duration_min'] >= 1) & (data_j['duration_min'] <= 60))
outlier_mask.sum()/len(data_j)

0.9812202822125979

In [9]:
data_j_outlier_masked = data_j[outlier_mask]


In [10]:
onehot_encode = pd.get_dummies(data_j_outlier_masked[['PULocationID', 'DOLocationID']], columns=['PULocationID', 'DOLocationID'])
features = onehot_encode.values
targets = data_j_outlier_masked['duration_min'].values

In [11]:
features.shape

(3009173, 515)

In [24]:
model = LinearRegression().fit(features, targets)

In [35]:
# with open('model.pkl', 'wb') as f:
#     pickle.dump(model, f)

In [14]:
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

In [47]:
np.sqrt(mean_squared_error(targets, model.predict(features)))

7.649304322688876

In [32]:
data_f['duration_min'] = (data_f['tpep_dropoff_datetime'] - data_f['tpep_pickup_datetime']) / pd.Timedelta(minutes=1)

In [33]:
outlier_mask_test = ((data_f['duration_min'] >= 1) & (data_f['duration_min'] <= 60))


In [37]:
data_f_masked = data_f[outlier_mask_test]

In [38]:
onehot_encode_test = pd.get_dummies(data_f_masked[['PULocationID', 'DOLocationID']], columns=['PULocationID', 'DOLocationID'])

onehot_encode_test = onehot_encode_test.reindex(columns=onehot_encode.columns, fill_value=0)

In [48]:
np.sqrt(mean_squared_error(data_f_masked['duration_min'].values, model.predict(onehot_encode_test.values)))

12341205804.804213